# Parse XML annotation file with X,Y coordinates and instance ID into a DataFrame

In [1]:
# def xml_to_df(xml_filepath):
#
#     tree = ET.parse(xml_filepath)
#     root = tree.getroot()
#     append_df = []
#     for index, Annotation in enumerate(root.iter("Annotation")):
#         for Region in Annotation.iter('Region'):
#             x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
#             y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
#             id = np.array([int(Region.get('Id'))])
#             classnames = index + 1
#             coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
#             df = pd.DataFrame(data = coord_dict)
#             df.ID = df.ID.astype(int)
#             append_df.append(df)
#     coord_df = pd.concat(append_df).reset_index(drop=True)
#     return(coord_df)

In [2]:
# coord_df = xml_to_df(r'\\fatherserverdw\kyuex\clue images\annotations\roi\2022-06-07 13.18.40.xml')
# coord_df

# Convert X,Y Coordinates to Binary Mask and Read and Resize Image (for ROI):

In [3]:
# import openslide
# import os
# os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()

# def df_to_image_mask(xml_filepath, ndpi_filepath,downsample_factor):
#     coord_df = xml_to_df(xml_filepath)
#     slide = openslide.open_slide(ndpi_filepath)
#     rgb_dim = slide.dimensions
#     print("Dimension of level 0 of the image is: ",rgb_dim)
#     slide_level_dim = slide.level_dimensions
#     num_levels = len(slide_level_dim)
#     print("Number of levels in this image are:",num_levels)
#     factors = slide.level_downsamples
#     print("Each level is downsampled by:",factors)
#     blank = np.zeros(rgb_dim, dtype = np.uint8) #white
#     for idx, row in coord_df.iterrows():
#         xx = row.X.astype('int')
#         yy = row.Y.astype('int')
#         xy = list(zip(xx,yy))
#         contours = np.array(xy)
#         mask = cv2.fillPoly(blank, pts=[contours.astype(int)], color=idx+125)
#     target_level = slide.get_best_level_for_downsample(downsample_factor)
#     target_dim = slide.level_dimensions[target_level]
#     rsf = [x/y for x,y in zip(rgb_dim,target_dim)]
#     image_resized = slide.read_region(location=(0,0),level=target_level,size=target_dim)
#     mask_resized = cv2.resize(mask,[_//round(rsf[0]) for _ in mask.shape],interpolation=cv2.INTER_NEAREST)
#     #cv2.imwrite(r"\\fatherserverdw\Kevin\\binarymask.jpg",mask_resized) #save binary mask
#     return image_resized,mask_resized

# Cropping RGB Image with Mask (for ROI):

In [4]:
# image,mask = df_to_image_mask(r'\\fatherserverdw\kyuex\clue images\annotations\roi\2022-06-07 13.18.40.xml',r'\\fatherserverdw\kyuex\clue images\2022-06-07 13.18.40.ndpi',10)

In [5]:
# def crop_final_mask(image,mask):
#     imagearr = np.array(image)
#     imagearr = imagearr[:,:,:3]
#     maskarr = np.array(mask)
#     maskrgb = np.repeat(maskarr[:,:,np.newaxis],3,axis=2)
#     for tissueID in range(1,np.max(maskrgb[:])): #from first tissue id to end
#         #masking
#         masktmp = mask==tissueID #boolean
#         masktmp = np.repeat(masktmp[:,:,np.newaxis],3,axis=2) #change shape to match shape of imagearr
#         final_image = np.multiply(imagearr,masktmp)
#         #crop
#         [x_crop,y_crop] = np.where(final_image[:,:,0]>0)
#         cropped_final_image = final_image[np.min(x_crop):np.max(x_crop),np.min(y_crop):np.max(y_crop)]
#         cropped_final_image = cropped_final_image[:,:,::-1]
#         return cropped_final_image

In [6]:
# crop_final_mask(image,mask)

# Workflow for Creating Cropped Mask and Image of Annotations (for multiple classes):
1. Convert XML file path to X,Y coordinate of contours.
2. Construct a mask with multiple annotation for the entire image, one mask with the same size of the image, with annotations # 1 to 12.
3. Label the binary version of constructed mask from #2 with unique pixel value for each annotation circle.
4. Iterate through the labeled binary mask to crop connected objects
5. Use the cropped coordinates from above #4 to simulataneously crop the constructed mask and the image.
6. Done!

In [7]:
# import all necessary modules required for the entire workflow:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import openslide
from PIL import Image
import cv2
from matplotlib import pyplot as plt
import skimage.measure
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()

In [8]:
# First, input xml_filepath and output a dataframe of X,Y coordinates in general. (can be used for ROI as well)
def xml_to_df(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    append_df = []
    for index, Annotation in enumerate(root.iter("Annotation")):
        for Region in Annotation.iter('Region'):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([int(Region.get('Id'))])
            classnames = index + 1
            coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data = coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df).reset_index(drop=True)
    return(coord_df)

In [9]:
# # Then, input xml_path to use xml_to_df function to output X,Y coordinates for each annotation per class:
# def coord_to_multiclass_df(xml_path):
#     coord_df = xml_to_df(xml_path)
#     coord_df = coord_df.drop(columns = ["ID"])
#     dict = {"corneum" : 1,"spinosum": 2,"hairshaft":3,"hairfollicle":4,"smoothmuscle":5,"oil":6,"sweat":7,"nerve":8,"bloodvessel":9,"ecm":10,"fat":11,"white":12}
#     coord_df = coord_df.replace({"ClassNames": dict})
#     coord_df.to_pickle(r'\\fatherserverdw\Kevin\coord_df.pkl') #save pkl
#     return coord_df

In [10]:
# # Then input original image and the coord_df to output the mask with unique annotations (1..N, N = 12 in this case):
# def create_mask_multi_annot(xml_path, image_path, downsample_factor = 10):
#     slide = openslide.open_slide(image_path)
#     target_level = slide.get_best_level_for_downsample(downsample_factor)
#     target_dim = slide.level_dimensions[target_level]
#     rsf = [x/y for x,y in zip(slide.dimensions,target_dim)] #resize factor
#     mask = np.zeros(target_dim, dtype = np.uint8)
#     iter_order = [2,10,5,4,6,11,7,9,8,12,3,1]
#     coord_df = coord_to_multiclass_df(xml_path) #use function above
#
#     for i in iter_order:
#         coord_df_tmp = coord_df[coord_df.ClassNames == i]
#         for idx, row in coord_df_tmp.iterrows():
#             xx = row.X.astype('int32')
#             yy = row.Y.astype('int32')
#             contours = np.array(list(zip(xx,yy)))
#             contours = contours/rsf[0]
#             class_number = row.ClassNames
#             mask = cv2.fillPoly(mask, pts=[contours.astype(int)], color=(int(class_number)))
#
#     maskimg = Image.fromarray(mask)
#     #maskimg.save(r'\\fatherserverdw\Kevin\binarymask_12class.jpg')
#     return mask

In [11]:
# # Input mask and create binary mask and then output label of connected regions:
# def create_binary_mask_label(xml_path, image_path):
#     mask = create_mask_multi_annot(xml_path, image_path, downsample_factor = 10)
#     binary_mask = mask > 0
#     binary_mask = binary_mask * 255 #255 if labeled (any class from 1 to 12), 0 if unlabeled
#     binary_mask_label = skimage.measure.label(binary_mask,connectivity = 2) #find connected regions, marked from 0 to 128 in the original image
#     return binary_mask_label #returns label of connected regions

In [12]:
# # Input label of connected regions and then output the final images and masks:
# def create_final_mask_image(xml_path, image_path, dstpath_mask, dstpath_image, downsample_factor = 10):
#     slide = openslide.open_slide(image_path)
#     target_level = slide.get_best_level_for_downsample(downsample_factor)
#     target_dim = slide.level_dimensions[target_level]
#     image = slide.read_region(location=(0,0),level=target_level,size=target_dim)
#     imagearr = np.array(image)
#     imagearr = imagearr[:,:,:3]
#     binary_mask_label = create_binary_mask_label(xml_path, image_path)
#     mask = create_mask_multi_annot(xml_path, image_path, downsample_factor = 10)
#     for idx,label in enumerate(range(1,np.max(binary_mask_label)+1)):
#         boo = binary_mask_label == label
#         boolabel = boo * label
#         loca = np.where(boolabel == label)
#         x = loca[0]
#         y = loca[1]
#         targetmask = mask[min(x):max(x),min(y):max(y)]
#         dstpth = dstpath_mask + str(idx)+'.png'
#         Image.fromarray(targetmask).save(dstpth)
#         targetim = imagearr[min(x):max(x),min(y):max(y),:]
#         dstpth1 = dstpath_image + str(idx)+'.png'
#         Image.fromarray(targetim).save(dstpth1)

In [13]:
# create_final_mask_image(xml_path = r'\\fatherserverdw\kyuex\clue images\annotations\12class\unhealthy_wrongbody\2022-06-07 13.18.40.xml', image_path = r'\\fatherserverdw\kyuex\clue images\2022-06-07 13.18.40.ndpi', downsample_factor = 10, dstpath_mask = r'\\fatherserverdw\Kevin\targetmask\\', dstpath_image = r'\\fatherserverdw\Kevin\targetimage\\')

In [14]:
# # Now use this function to create final mask images of other XML's:
# create_final_mask_image(xml_path = r'\\fatherserverdw\kyuex\clue images\annotations\12class\2022-06-07 14.34.59.xml', image_path = r'\\fatherserverdw\kyuex\clue images\2022-06-07 14.34.59.ndpi', dstpath_mask = r'\\fatherserverdw\kyuex\clue images\annotations\12class\DLtrainingset_clue\2022-06-07 14.34.59\mask\\', dstpath_image = r'\\fatherserverdw\kyuex\clue images\annotations\12class\DLtrainingset_clue\2022-06-07 14.34.59\image\\', downsample_factor = 10)

In [15]:
# create_final_mask_image(xml_path = r'\\fatherserverdw\kyuex\clue images\annotations\12class\2022-06-07 14.32.24.xml', image_path = r'\\fatherserverdw\kyuex\clue images\2022-06-07 14.32.24.ndpi', dstpath_mask = r'\\fatherserverdw\kyuex\clue images\annotations\12class\DLtrainingset_clue\2022-06-07 14.32.24\mask\\', dstpath_image = r'\\fatherserverdw\kyuex\clue images\annotations\12class\DLtrainingset_clue\2022-06-07 14.32.24\image\\', downsample_factor = 10)

In [16]:
# create_final_mask_image(xml_path = r'\\fatherserverdw\kyuex\clue images\annotations\12class\2022-06-07 14.37.19.xml', image_path = r'\\fatherserverdw\kyuex\clue images\2022-06-07 14.37.19.ndpi', dstpath_mask = r'\\fatherserverdw\kyuex\clue images\annotations\12class\DLtrainingset_clue\2022-06-07 14.37.19\mask\\', dstpath_image = r'\\fatherserverdw\kyuex\clue images\annotations\12class\DLtrainingset_clue\2022-06-07 14.37.19\image\\', downsample_factor = 10)

In [17]:
# create_final_mask_image(xml_path = r'\\fatherserverdw\kyuex\clue images\annotations\12class\2022-06-07 14.27.44.xml', image_path = r'\\fatherserverdw\kyuex\clue images\2022-06-07 14.27.44.ndpi', dstpath_mask = r'\\fatherserverdw\kyuex\clue images\annotations\12class\DLtrainingset_clue\2022-06-07 14.27.44\mask\\', dstpath_image = r'\\fatherserverdw\kyuex\clue images\annotations\12class\DLtrainingset_clue\2022-06-07 14.27.44\image\\', downsample_factor = 10)

In [18]:
coord_df_1 = xml_to_df(r"\\fatherserverdw\kyuex\clue images\annotations\12class\2022-06-07 14.34.59.xml")
coord_df_1 = coord_df_1.drop(columns = ["ID"])
dict = {"corneum" : 1,"spinosum": 2,"hairshaft":3,"hairfollicle":4,"smoothmuscle":5,"oil":6,"sweat":7,"nerve":8,"bloodvessel":9,"ecm":10,"fat":11,"white":12}
coord_df_1 = coord_df_1.replace({"ClassNames": dict})
coord_df_1.to_pickle(r'\\fatherserverdw\Kevin\coord_df.pkl')

In [19]:
coord_df_1 = pd.read_pickle(r'\\fatherserverdw\Kevin\coord_df.pkl')
coord_df_1

,ClassNames,X,Y
0,1,"[19495, 19513, 19517, 19521, 19526, 19526, 195...","[22413, 22409, 22409, 22405, 22405, 22400, 223..."
1,1,"[26109, 26104, 26100, 26096, 26096, 26091, 260...","[19866, 19866, 19861, 19861, 19857, 19857, 198..."
2,1,"[8366, 8367, 8371, 8373, 8374, 8379, 8380, 838...","[24856, 24856, 24857, 24858, 24860, 24861, 248..."
3,1,"[18519, 18518, 18518, 18517, 18517, 18516, 185...","[21348, 21348, 21349, 21349, 21350, 21351, 213..."
4,1,"[15897, 15897, 15897, 15897, 15897, 15897, 158...","[22538, 22540, 22541, 22542, 22543, 22544, 225..."
...,...,...,...
71,12,"[10559, 10558, 10556, 10553, 10549, 10544, 105...","[24644, 24644, 24642, 24641, 24635, 24630, 246..."
72,12,"[9814, 9814, 9813, 9813, 9812, 9812, 9811, 980...","[19642, 19643, 19643, 19644, 19644, 19645, 196..."
73,12,"[17721, 17720, 17720, 17719, 17719, 17719, 177...","[18463, 18463, 18464, 18464, 18465, 18467, 184..."
74,12,"[8673, 8671, 8669, 8667, 8665, 8662, 8660, 865...","[25096, 25096, 25096, 25096, 25096, 25096, 250..."


In [20]:
# slide = openslide.open_slide(r"\\fatherserverdw\kyuex\clue images\2022-06-07 14.34.59.ndpi")
# target_level = slide.get_best_level_for_downsample(10)
# target_dim = slide.level_dimensions[target_level]
# rsf = [x/y for x,y in zip(slide.dimensions,target_dim)] #resize factor
# image = slide.read_region(location=(0,0),level=target_level,size=target_dim)
# imagearr = np.array(image)
# imagearr = imagearr[:,:,:3]
# mask = np.zeros((target_dim[1],target_dim[0]), dtype = np.uint8)
#
# iter_order = [2,10,5,4,6,11,7,9,8,12,3,1]
# for i in iter_order:
#     coord_df_1tmp = coord_df_1[coord_df_1.ClassNames == i]
#     for idx, row in coord_df_1tmp.iterrows():
#         xx = row.X.astype('int32')
#         yy = row.Y.astype('int32')
#         contours = np.array(list(zip(xx,yy)))
#         contours = contours/rsf[0]
#         class_number = row.ClassNames
#         mask = cv2.fillPoly(mask, pts=[contours.astype(int)], color=(int(class_number)))
#
# # maskimg = Image.fromarray(mask)
# # maskimg.show()
# # maskimg.save(r'\\fatherserverdw\Kevin\binarymask_12class.jpg') <- binary mask

In [21]:
# slide = openslide.open_slide(r"\\fatherserverdw\kyuex\clue images\2022-06-07 14.34.59.ndpi")
# target_level = slide.get_best_level_for_downsample(10)
# target_dim = slide.level_dimensions[target_level]
# rsf = [x/y for x,y in zip(slide.dimensions,target_dim)] #resize factor
# image = slide.read_region(location=(0,0),level=target_level,size=target_dim)
# imagearr = np.array(image)
# imagearr = imagearr[:,:,:3]
#
# binary_mask = mask > 0
# binary_mask_label = skimage.measure.label(binary_mask,connectivity = 2) #find connected regions, marked from 0 to 128 in the original image
#
# for idx,label in enumerate(range(1,np.max(binary_mask_label)+1)):
#     boo = binary_mask_label == label
#     boolabel = boo * label
#     loca = np.where(boolabel == label)
#     x = loca[0]
#     y = loca[1]
#     targetmask = mask[min(x):max(x),min(y):max(y)]
#     dstpth = r'\\fatherserverdw\Kevin\targetmask\\' + str(idx)+'.png'
#     Image.fromarray(targetmask).save(dstpth)
#
#     targetim = imagearr[min(x):max(x),min(y):max(y),:]
#     dstpth1 = r'\\fatherserverdw\Kevin\targetimage\\' + str(idx)+'.png'
#     Image.fromarray(targetim).save(dstpth1)
#
#     target_mask_1 = targetmask > 0
#     target_mask_1 = np.repeat(target_mask_1[:,:,np.newaxis],3,axis=2)
#
#     final_image = np.multiply(targetim,target_mask_1)
#     dstpth2 = r'\\fatherserverdw\Kevin\finaltargetimage\\' + str(idx)+'.png'
#     Image.fromarray(final_image).save(dstpth2)

In [22]:
# imsrc = r"\\fatherserverdw\Kevin\targetimage"
# imdst = os.path.join(imsrc,'immasked')
# if not os.path.exists(imdst):
#     os.mkdir(imdst)
# msksrc = r"\\fatherserverdw\Kevin\targetmask"
# imlist = [_ for _ in os.listdir(imsrc) if _.endswith('.png')]
# for im in imlist:
#     imarr = np.array(Image.open(os.path.join(imsrc,im)))
#     mskarr = np.array(Image.open(os.path.join(msksrc,im)))
#     mskarr = mskarr>0
#     mskarr3d = np.repeat(mskarr[:,:,np.newaxis],3,axis=2)
#     immasked = np.multiply(imarr,mskarr3d)
#     Image.fromarray((immasked)).save(os.path.join(imdst,im))


# Randomly (from a gaussian distribution) place all images from the output images on top of a 5000 x 5000 image tile. Do the same for the mask:
It looks like the files in the path: fatherserverdw\Q\research\images\skin_aging\deeplab_trainingset\v11\training\big_tiles


In [24]:
import matplotlib.pyplot as plt
import scipy.stats as stats

image_tile = np.zeros((5000,5000)) #black image tile
image_tile = Image.fromarray(image_tile)
# pick one file to work with first:
number_of_files = len(os.listdir(r'\\fatherserverdw\Kevin\targetimage\immasked'))
rand_file_num = np.random.randint(1,number_of_files,size = number_of_files+1)
lower = 0
upper = 5000
mu = 2500
sigma = 1250
rand_coord = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc = mu, scale = sigma)
# plt.hist(rand_coord_pair.rvs(10000))
# plt.show()
rand_coord_x = np.round(rand_coord.rvs(10000))
rand_coord_x = [int(x) for x in rand_coord_x]
rand_coord_y = np.round(rand_coord.rvs(10000))
rand_coord_y = [int(y) for y in rand_coord_y]

for idx in np.nditer(rand_file_num):
    image = Image.open(r'\\fatherserverdw\Kevin\targetimage\immasked\\' + str(rand_file_num[idx]) + '.png')
    x = rand_coord_x[idx]
    y = rand_coord_y[idx]
    image_tile.paste(image,(x,y))
image_tile.show() #change to rgb

# for each file number from the random distribution
    # for each x,y coordinate pair, place image on the black image tile
        # if overlap, try next coordinates

In [ ]:
ra = np.array(image)
ra

# Image Registration of Two Images:

In [ ]:
# import pyelastix as pe
# import imageio.v2 as imageio
# import os
# import time
#
# path = r'\\fatherserverdw\Q\research\images\CLUE\3D study\he\4xr_jpg'
# image_files_path = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.jpg')]
#
# image1 = imageio.imread(image_files_path[0])
# image2 = imageio.imread(image_files_path[1])
# image1 = image1[:,:,1].astype('float32')
# image2 = image2[:,:,1].astype('float32')
# print(image1.shape == image2.shape)

In [ ]:
# def pad_images_to_same_size(images):
#     width_max = 0
#     height_max = 0
#     for img in images: #pad image to the largest image
#         h, w = img.shape[:2]
#         width_max = max(width_max, w)
#         height_max = max(height_max, h)
#
#     images_padded = []
#     for img in images:
#         h, w = img.shape[:2]
#         diff_vert = height_max - h
#         pad_top = diff_vert//2
#         pad_bottom = diff_vert - pad_top
#         diff_hori = width_max - w
#         pad_left = diff_hori//2
#         pad_right = diff_hori - pad_left
#         img_padded = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=255)
#         assert img_padded.shape[:2] == (height_max, width_max)
#         images_padded.append(img_padded)
#
#     return images_padded

# image1, image2 = pad_images_to_same_size([image1,image2])
# print(image1.shape == image2.shape)
#
# padded_image1 = Image.fromarray(image1)
# padded_image1 = padded_image1.convert("L")
# #padded_image1.save(r'\\fatherserverdw\Kevin\paddedimage1.jpg')
#
# padded_image2 = Image.fromarray(image2)
# padded_image2 = padded_image2.convert("L")
# #padded_image2.save(r'\\fatherserverdw\Kevin\paddedimage2.jpg')

In [ ]:
# params = pe.get_default_params()
# params.MaximumNumberOfIterations = 200
# params.NumberOfResolutions = 3
#
# start = time.time()
# image3, field = pe.register(image1,image2,params)
# end = time.time()
# print("time it took to register: "+  str(end-start) + " seconds")
#
# img3 = Image.fromarray(image3)
# img3.show()
# img3 = img3.convert("L")
# img3.save(r'\\fatherserverdw\Kevin\image3.jpg')

In [ ]:
# overlay image 3 on top of image 1 and image 2 to see how well the image 2 is rotated in reference to image 1:
# alpha, beta = 0.5, 0.5
# overlay_img1_img3 = cv2.addWeighted(image1,alpha,image3,beta,0.0)
# img_overlay = Image.fromarray(overlay_img1_img3)
# img_overlay.show()
# img_overlay = img_overlay.convert("L")
# img_overlay.save(r'\\fatherserverdw\Kevin\overlay.jpg')
#
# another method:
# difference = image1 - image3
# difference = difference.astype(np.int)
# difference_df = pd.DataFrame(difference)
# difference_df = difference_df[difference_df>0]
# difference_df = difference_df.fillna(0)
# diff_counts = np.unique(difference_df,return_counts=True)[1]
# sum_diff = np.sum(diff_counts) - diff_counts[0]
# print("{} pixel values out of {} total pixel values are different".format(sum_diff,image1.shape[0]*image1.shape[1]))
# print("{}% pixel values are different".format(sum_diff*100/(image1.shape[0]*image1.shape[1])))

In [ ]:
# # stack three x,y values (2d image) to 1 2d Image (x,y,3)
# ig1 = imageio.imread(r'\\fatherserverdw\Q\research\images\CLUE\3D study\he\4xr_jpg\z0001_1C1.jpg')
# ig2 = imageio.imread(r'\\fatherserverdw\Q\research\images\CLUE\3D study\he\4xr_jpg\z0004_4C1.jpg')
# ig3 = imageio.imread(r'\\fatherserverdw\Q\research\images\CLUE\3D study\he\4xr_jpg\z0007_7C1.jpg')
# #pad before:
# ig1, ig2, ig3 = pad_images_to_same_size([ig1,ig2,ig3])
# ig1 = cv2.cvtColor(ig1,cv2.COLOR_RGB2GRAY)
# ig2 = cv2.cvtColor(ig2,cv2.COLOR_RGB2GRAY)
# ig3 = cv2.cvtColor(ig3,cv2.COLOR_RGB2GRAY)
# # all img shape is (6264, 5352)
# rgb = np.stack([ig1,ig2,ig3],axis=2)
# rgbimg = Image.fromarray(rgb)
# rgbimg.show()
# rgbimg.save(r'\\fatherserverdw\Kevin\rgbimg.jpg')

In [ ]:
# grayimg = cv2.cvtColor(np.array(rgbimg),cv2.COLOR_RGB2GRAY)
# grayimg = grayimg.astype(np.uint8)
# plt.imshow(grayimg)
#
# params = pe.get_default_params()
# params.MaximumNumberOfIterations = 200
# params.NumberOfResolutions = 3
#
# start = time.time()
# fin_img, field = pe.register(grayimg,ig1,params)
# end = time.time()
# print("time it took to register: "+  str(end-start) + " seconds")
#
# finimg = Image.fromarray(fin_img)
# finimg.show()
# img3 = img3.convert("L")
# img3.save(r'\\fatherserverdw\Kevin\image3.jpg')

In [ ]:
# import SimpleITK as sitk
#
# elastixImageFilter = sitk.ElastixImageFilter()
# elastixImageFilter.SetFixedImage(sitk.ReadImage(r'\\fatherserverdw\Kevin\rgbimg.jpg'))
# elastixImageFilter.SetMovingImage(sitk.ReadImage(r'\\fatherserverdw\Q\research\images\CLUE\3D study\he\4xr_jpg\z0001_1C1.jpg'))
#
# parameterMapVector = sitk.VectorOfParameterMap()
# parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
# parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
# elastixImageFilter.SetParameterMap(parameterMapVector)
#
# elastixImageFilter.Execute()
# sitk.WriteImage(elastixImageFilter.GetResultImage())